<div style="text-align: center;">
  <h2><b>Ho Chi Minh University of Science</b></h2>
  <h3><b>Mathematics & Computer Science Faculty</b></h3>
  <img src='https://th.bing.com/th/id/R.7298e6a7f00d000e22456c9977085835?rik=DqP9BjBZsGKPxw&pid=ImgRaw&r=0' width="250px">
  <h1><b>COURSE PROJECT: PYTHON FOR DATA SCIENCE</b></h1>
  <ul style="list-style-type: none; padding: 0;">
    <li>
      <table style="margin: 0 auto; border-collapse: collapse; width: 50%;">
        <thead>
          <tr>
            <th style="border: 1px solid black; padding: 8px;">Full Name</th>
            <th style="border: 1px solid black; padding: 8px;">Student ID</th>
          </tr>
        </thead>
        <tbody>
          <tr>
            <td style="border: 1px solid black; padding: 8px;">Nguyễn Lê Lâm Phúc</td>
            <td style="border: 1px solid black; padding: 8px;">22280066</td>
          </tr>
          <tr>
            <td style="border: 1px solid black; padding: 8px;">Mạc Minh Phúc</td>
            <td style="border: 1px solid black; padding: 8px;">22280065</td>
          </tr>
          <tr>
            <td style="border: 1px solid black; padding: 8px;">Trần Đại Lộc</td>
            <td style="border: 1px solid black; padding: 8px;">22280053</td>
          </tr>
          <tr>
            <td style="border: 1px solid black; padding: 8px;">Nguyễn Lê Đăng Khoa</td>
            <td style="border: 1px solid black; padding: 8px;">22280047</td>
          </tr>
        </tbody>
      </table>
    </li>
  </ul>
</div>


Link code: https://colab.research.google.com/drive/1gW6h87Hf4Glf2dqNvZFF2mECtX2--zyM#scrollTo=TXyfP-P3oLfp

##**Question 2:** Chatbot Development

### Import libraries

In [ ]:
!pip install python-dotenv

In [ ]:
!pip install bs4

In [ ]:
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
import os
from bs4 import BeautifulSoup
import requests
import dotenv

### Crawling data

In [ ]:
# Get the website URL
url='https://www.presight.io/privacy-policy.html'
result= requests.get(url)
soup=BeautifulSoup(result.text,'html.parser')

In [ ]:
def parse_elements(elements):
    """
    Parse elements from HTML and structure them into a JSON format.

    Args:
        elements (list): List of HTML elements to parse.

    Returns:
        dict: Parsed data structured as a JSON.
    """
    data = {"PRIVACY POLICY": []}
    prev_element = None
    current_header = None

    # Skip unnecessary part, find and crawl only relevant data
    for element in elements[3:]:
        if element.name == 'i':
            handle_italic_element(element, current_header, prev_element)
        elif element.name == 'h2':
            current_header = handle_header_element(element, prev_element, current_header, data)
        elif element.name in ['p', 'li']:
            handle_content_element(element, current_header)

        prev_element = element

    if current_header:
        data["PRIVACY POLICY"].append(current_header)

    return data

def handle_italic_element(element, current_header, prev_element):
    """
    Handle italic elements (<i> tags) and add them as subheaders to construct a more reasonable JSON file.

    Args:
        element (bs4.element.Tag): The <i> element to process.
        current_header (dict): The current header being processed.
        prev_element (bs4.element.Tag): The previous element.
    """
    if current_header:
        current_header["Subheaders"].append({
            "Title": element.text.strip(),
            "Content": ""
        })
    else:
        current_header = {
            "Title": prev_element.text.strip(),
            "Subheaders": [
                {"Title": element.text.strip(), "Content": ""}
            ]
        }

def handle_header_element(element, prev_element, current_header, data):
    """
    Handle header elements (<h2> tags) and structure them accordingly.

    Args:
        element (bs4.element.Tag): The <h2> element to process.
        prev_element (bs4.element.Tag): The previous element.
        current_header (dict): The current header being processed.
        data (dict): The main data dictionary.

    Returns:
        dict: Updated current header.
    """
    if element.text == "Automated Edit Checks":
        if current_header:
            current_header["Subheaders"].append({
                "Title": element.text.strip(),
                "Content": ""
            })
    elif prev_element and prev_element.name == 'h2':
        if current_header:
            current_header["Subheaders"].append({
                "Title": element.text.strip(),
                "Content": ""
            })
        else:
            current_header = {
                "Title": prev_element.text.strip(),
                "Subheaders": [
                    {"Title": element.text.strip(), "Content": ""}
                ]
            }
    else:
        if current_header:
            data["PRIVACY POLICY"].append(current_header)
        current_header = {"Title": element.text.strip(), "Content": "", "Subheaders": []}

    return current_header

def handle_content_element(element, current_header):
    """
    Handle content elements (<p> and <li> tags) and add content to the appropriate section.

    Args:
        element (bs4.element.Tag): The content element to process.
        current_header (dict): The current header being processed.
    """
    if current_header:
        if current_header["Subheaders"]:
            current_header["Subheaders"][-1]["Content"] += element.text.strip() + ("; " if element.name == 'li' else " ")
        else:
            current_header["Content"] += element.text.strip() + ("; " if element.name == 'li' else " ")

def split_first_element(data):
    """
    Splits the first element in the PRIVACY POLICY into two distinct elements.

    Args:
        data (dict): The original JSON data.

    Returns:
        dict: Updated JSON data with the first element split.
    """
    result = {"PRIVACY POLICY": []}
    for entry in data["PRIVACY POLICY"]:
        if entry["Title"].startswith("Last updated"):
            result["PRIVACY POLICY"].append({
                "Title": "Update date",
                "Content": entry["Title"],
                "Subheaders": []
            })
            result["PRIVACY POLICY"].append({
                "Title": "Commitment",
                "Content": entry["Content"],
                "Subheaders": entry["Subheaders"]
            })
        else:
            result["PRIVACY POLICY"].append(entry)

    return result

In [ ]:
elements = soup.find_all(['p', 'h2', 'li', 'i'])
parsed_data = parse_elements(elements)
result = split_first_element(parsed_data)
#Print first 3 elements in the JSON file
print(json.dumps(result["PRIVACY POLICY"][:3], indent=4, ensure_ascii=False))

[
    {
        "Title": "Update date",
        "Content": "Last updated 15 Sep 2023",
        "Subheaders": []
    },
    {
        "Title": "Commitment",
        "Content": "At Presight, we are committed to protecting the privacy of our customers and visitors to our website. This Privacy Policy explains how we collect, use, and disclose information about our customers and visitors. ",
        "Subheaders": []
    },
    {
        "Title": "Information Collection and Use",
        "Content": "We collect several different types of information for various purposes to provide and improve our Service to you. ",
        "Subheaders": []
    }
]


In [ ]:
# Write the JSON data to a file
output_file = "privacy_policy.json"

with open(output_file, "w", encoding="utf-8") as file:
    json.dump(result, file, indent=4, ensure_ascii=False)

print(f"Data has been written to {output_file}")

Data has been written to privacy_policy.json


### Embedding data and developing chatbot

#### Load API Key

In [ ]:
# Load API key from environment variables
dotenv.load_dotenv()
api_key_2 = os.getenv('CHAT_API_KEY')

if not api_key_2:
    raise ValueError("API key not found. Please set the 'CHAT_API_KEY' environment variable.")

#### Develop chatbot

In [ ]:
class PrivacyPolicyAssistant:
    def __init__(self, model_name="all-MiniLM-L6-v2", title_weight=2, thresh_hold=0.5, genai_api_key="", file_path="privacy_policy.json", max_file_size=1048576):
        """
        Initializes the PrivacyPolicyAssistant.

        Args:
            model_name (str): Name of the transformer model for embeddings.
            title_weight (float): Weight for title embeddings in relevance calculations.
            thresh_hold (float): Similarity threshold for relevance.
            genai_api_key (str): API key for the generative AI model.
            file_path (str): Path to the privacy policy JSON file.
            max_file_size (int): Maximum size for saved history files (in bytes).

        Returns:
            None
        """
        self.title_weight = title_weight
        self.thresh_hold = thresh_hold
        self.documents = []
        self.embeddings = None
        self.model = SentenceTransformer(model_name)
        genai.configure(api_key=genai_api_key)
        self.generative_model = genai.GenerativeModel("gemini-1.5-flash")
        self.file_path = file_path
        self.max_file_size = max_file_size
        self._prepare_and_embed_data()

    def _load_json(self):
        """
        Loads the privacy policy JSON data from the specified file.
        """
        with open(self.file_path, "r", encoding="utf-8") as file:
            return json.load(file)

    def _prepare_and_embed_data(self):
        """
        Processes the privacy policy sections into documents and computes their embeddings.
        Combines titles, contents, and subheaders to generate meaningful embeddings
        for similarity-based querying.
        """
        documents = []
        embeddings = []
        data = self._load_json()["PRIVACY POLICY"]

        for section in data:
            title = section.get("Title", "")
            content = section.get("Content", "")
            subheaders = section.get("Subheaders", [])
            if not subheaders:
              # Handles sections without subheaders
                combined_text = f"{title} {content}"
                documents.append({"text": combined_text, "context": title})
                # Adding weight to title to meet users' basic needs for concepts and definitions, avoiding information noise
                embeddings.append(self.model.encode(title) * self.title_weight + self.model.encode(content))
            else:
              # Handles sections with subheaders
                for subheader in subheaders:
                    sub_title = subheader.get("Title", "")
                    sub_content = subheader.get("Content", "")
                    documents.append({"text": sub_content, "context": sub_title})
                    embeddings.append(self.model.encode(sub_title) * self.title_weight + self.model.encode(sub_content))

                subheader_contents = " ".join([f"{sh['Title']}: {sh['Content']}" for sh in subheaders])
                header_combined_text = f"{title} {content} {subheader_contents}"
                documents.append({"text": header_combined_text, "context": title})
                embeddings.append(self.model.encode(title) * self.title_weight + self.model.encode(header_combined_text))

        self.documents = documents
        self.embeddings = np.array(embeddings)

    def save_history(self, file_path, user_message, bot_response):
      """
        Saves a user-bot conversation history into a JSON file.

        Args:
            file_path (str): Path to the file where history will be saved.
            user_message (str): User's query or message.
            bot_response (str): Generated response to the user's query.

        Returns:
            None
        """
      new_exchange = {"user": user_message, "bot": bot_response}
      new_content = json.dumps(new_exchange, indent=4)

      if os.path.exists(file_path):
          current_size = os.path.getsize(file_path)
          write_mode = "a"
      else:
          current_size = 0
          write_mode = "w"

      new_content_size = len(new_content.encode("utf-8"))
      if current_size + new_content_size > self.max_file_size:
          print(f"Alert: File size limit of {self.max_file_size} bytes exceeded. Save aborted.")
          return

      else:
            if write_mode == "a":
                # If the file exists and is not empty, append new data to the existing JSON array
              with open(file_path, "r+", encoding="utf-8") as f:
                  f.seek(0, os.SEEK_END)
                  f.seek(f.tell() - 1, os.SEEK_SET) # Adjusts the file pointer to overwrite the closing bracket
                  f.truncate()
                  f.write(",\n")
                  f.write(new_content)
                  f.write("\n]")
            else:
                with open(file_path, write_mode, encoding="utf-8") as f:
                    f.write("[\n")
                    f.write(new_content)
                    f.write("\n]")

    def find_most_relevant(self, query):
        """
        Finds the most relevant privacy policy section for the given query using cosine similarity.

        Args:
            query (str): The user's query for which relevance is evaluated.

        Returns:
            tuple: A dictionary of the most relevant document and its similarity score.
        """
        query_embedding = self.model.encode(query)
        similarities = cosine_similarity([query_embedding], self.embeddings).flatten()
        most_relevant_idx = np.argmax(similarities)
        return self.documents[most_relevant_idx], similarities[most_relevant_idx]

    def make_request(self, query):
        """
        Generates a response to the user's query based on the most relevant privacy policy section or fallback logic.
        Args:
            query (str): The user's query for the assistant.
        Returns:
            None
        """
        result, similarity = self.find_most_relevant(query)
        if similarity > self.thresh_hold:
            prompt = (f"You are an assistant about company privacy policy. Answer {query} based on this mock answer {result}.")
        else:
          # Handle cases where questions contain context about the information above
          if "this" in query or "that" in query or "above" in query:
            history = json.load(open("conversation_history.json","r"))
            if len(history) > 0:
              current_ques=history[-1]["user"]
              current_ans=history[-1]["bot"]
              prompt = (f"You are an assitant about company privacy policy. Your task is to answer {query} which is sequenced from {current_ques} and your current answer {current_ans}")
            else:
              print("Please give me further information about your context!!!")
          else:
            # Provides a generic response with a link for more information
            prompt = (f"You are an assitant about company privacy policy. Your task is to answer {query} and then told \"If you want to get further information about {query}, please visit our website https://www.presight.io/privacy-policy.html\"")

        try:
            response = self.generative_model.generate_content(prompt)
            print(response.text)
            self.save_history("conversation_history.json",query,response.text)

        except Exception as e:
            print(f"Error generating response: {e}")


#### Example

In [ ]:
# Usage example
assistant = PrivacyPolicyAssistant(genai_api_key=api_key_2)
query = "What is Personal Data?"
assistant.make_request(query)

Personal Data, as used in our Service's privacy policy, refers to information that can be used to contact or identify you.  This includes, but is not limited to, your email address, first and last name, phone number, address (including state, province, ZIP/Postal code, and city), and Cookies and Usage Data.



In [ ]:
# Query with previous context
query_2 = "Explain further about this?"
assistant.make_request(query_2)

Okay, let's break down what "Personal Data" encompasses in more detail, following up on the definition provided:

**Personal Data, as used in our Service's privacy policy, refers to information that can be used to contact or identify you.  This includes, but is not limited to, your email address, first and last name, phone number, address (including state, province, ZIP/Postal code, and city), and Cookies and Usage Data.**

Let's examine each component:

* **Information that can be used to contact or identify you:** This is the core principle.  Any information that, alone or when combined with other data, could reasonably be used to pinpoint your identity or get in touch with you is considered personal data.  The key is "reasonably."  For example, a publicly available username might not be considered personal data on its own, but if combined with your email address, it significantly increases the likelihood of identification.

* **Email address:**  This is a direct and primary identifi

In [ ]:
# Query about information not in the data
query_3 = "What is policy?"
assistant.make_request(query_3)

A privacy policy is a legal statement that explains how a company collects, uses, and protects the personal information of its users or customers.  It outlines what data is collected, why it's collected, how it's used, who it might be shared with, and what security measures are in place to protect it.  It also often details individuals' rights regarding their data, such as the right to access, correct, or delete their information.

If you want to get further information about what constitutes a privacy policy and the specifics of our policy, please visit our website: https://www.presight.io/privacy-policy.html



In [ ]:
# Yes/ No question
query_4 = "Do I need to provide my Email Address in Personal Data?"
assistant.make_request(query_4)

Based on the provided text, the answer is **yes**.  The privacy policy explicitly states that an email address is included in the types of personally identifiable information ("Personal Data") that may be requested.



In [ ]:
# Print out the history conversation
json.load(open("conversation_history.json","r"))

[{'user': 'What is Personal Data?',
  'bot': "Personal Data, as used in our Service's privacy policy, refers to information that can be used to contact or identify you.  This includes, but is not limited to, your email address, first and last name, phone number, address (including state, province, ZIP/Postal code, and city), and Cookies and Usage Data.\n"},
 {'user': 'Explain further about this?',
  'bot': 'Okay, let\'s break down what "Personal Data" encompasses in more detail, following up on the definition provided:\n\n**Personal Data, as used in our Service\'s privacy policy, refers to information that can be used to contact or identify you.  This includes, but is not limited to, your email address, first and last name, phone number, address (including state, province, ZIP/Postal code, and city), and Cookies and Usage Data.**\n\nLet\'s examine each component:\n\n* **Information that can be used to contact or identify you:** This is the core principle.  Any information that, alone o